In [64]:
import numpy as np
from scipy import signal

# Часть первая

### Реализовать ```adaptrls```

In [65]:
def adaptrls(x : np.ndarray, d : np.ndarray, p : int = 16, Lambda : int = 1, alpha : float = 1e4):
    N = len(x) - p
    w = np.zeros(p)
    y = np.zeros(N)
    e = np.zeros(N)
    R_inv = alpha * np.eye(p)
    for k in range(N):
        xx = np.array([x[k + p - 1 - i] for i in range(p)])
        y[k] = np.dot(xx.T, w)
        e[k] = d[k + p - 1] - y[k]
        K = (np.dot(R_inv, xx)) / (Lambda + np.dot(np.dot(xx.T, R_inv), xx))
        R_inv = (R_inv - np.dot(np.outer(K, xx.T), R_inv)) / Lambda
        w = w + K * e[k]
    return w, e

### Сгенерировать вектор характеристик, случ. вектор x, получить выходной вектор

In [66]:
p = 10
N = 400
h = np.random.normal(1, 2, p)
x = np.random.normal(0, 0.5, N)
x = np.hstack((np.zeros(p - 1), x))

d = signal.lfilter(h, 1, x)

In [67]:
w, e = adaptrls(x, d, p)

In [68]:
print(h)
print(w)
print(e)

[2.06451372 1.05962417 3.14627672 5.21843789 2.84506542 1.95745549
 3.13437488 2.71873193 1.86934144 2.0992254 ]
[2.06451141 1.05962281 3.1462735  5.21843283 2.84506251 1.95745322
 3.13437163 2.71872913 1.86933939 2.09922309]
[ 1.07160498e+00  5.49780636e-01  1.63191643e+00  2.70117490e+00
  1.44032060e+00  9.15795499e-01  1.40086201e+00  8.29378571e-01
 -2.95326176e-01 -7.62559986e-01 -2.43239610e+00 -1.02448396e-02
 -2.72644263e-03  1.48672989e-04 -9.30448965e-04 -1.40827042e-03
 -2.95966352e-04 -3.60166399e-05  7.15424640e-04  8.56819606e-04
  6.10774598e-04  7.53422819e-04  1.04094067e-03  6.62869827e-04
  1.31752875e-04  2.53321303e-04 -4.83134104e-04 -1.60092454e-04
 -1.47201442e-04 -3.63885068e-04 -8.68246219e-05 -1.91967348e-06
 -2.06646487e-04 -1.44263738e-05  9.39177349e-05 -2.16843213e-05
  6.89944926e-05 -1.76939509e-04 -3.17054880e-04 -2.46346490e-04
 -2.96938068e-04 -3.15393783e-04 -1.97341747e-04 -1.58190490e-04
 -1.70637440e-04 -1.19275595e-04 -6.80097053e-05 -4.5251104